In [1]:
%cd ..
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role
import tarfile
import boto3
import os
from pathlib import Path

/home/ec2-user/SageMaker/gender-classification


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[04/26/25 02:51:34] INFO     Found credentials from IAM Role:                                   ]8;id=398703;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=966945;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
def compress_model(model_path):
    filename = os.path.basename(model_path)
    dir_name = os.path.dirname(model_path)
    archived_filename = f'{filename.replace(".pt", "")}.tar.gz'
    with tarfile.open(archived_filename, 'w:gz') as tar:
        tar.add(model_path, arcname=filename)
    return archived_filename

def push_model_to_s3(model_path, bucket):
    model_filename = compress_model(model_path)
    s3 = boto3.client('s3')
    s3.upload_file(
        Filename=model_filename,
        Bucket=bucket,
        Key=f'models/{model_filename}'
    )
    Path(model_filename).unlink(missing_ok=True)
    return f's3://{bucket}/models/{model_filename}'

In [3]:
model_s3_path = push_model_to_s3(model_path='./gender_classifier_250418.pt',
                                 bucket='hieunt-sagemaker-gender-classification')
model_s3_path

[04/26/25 02:52:06] INFO     Found credentials from IAM Role:                                   ]8;id=768763;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=195860;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

's3://hieunt-sagemaker-gender-classification/models/gender_classifier_250418.tar.gz'

In [4]:
role = get_execution_role()

pytorch_model = PyTorchModel(model_data=model_s3_path,
                             role=role,
                             entry_point='src/inference.py',
                             framework_version='2.2.0',
                             name='gender-classification',
                             py_version='py310')

In [5]:
predictor = pytorch_model.deploy(instance_type='ml.t2.medium', initial_instance_count=1, endpoint_name='gender-classifier')

[04/26/25 02:52:36] INFO     Repacking model artifact                                                  ]8;id=224937;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/model.py\model.py]8;;\:]8;id=531807;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/model.py#820\820]8;;\
                             (s3://hieunt-sagemaker-gender-classification/models/gender_classifier_250             
                             418.tar.gz), script artifact (None), and dependencies ([]) into single                
                             tar.gz file located at                                                                
                             s3://sagemaker-ap-southeast-1-406829288865/gender-classification/model.ta             
                             r.gz. This may take some time depending on model size...                              

[04/26/25 02:53:15] INFO     Creating model with name: gender-classification                        ]8;id=295379;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=545394;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

[04/26/25 02:53:16] WARNING  Using already existing model: gender-classification                    ]8;id=135986;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=415391;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4105\4105]8;;\

                    INFO     Creating endpoint-config with name gender-classifier                   ]8;id=856871;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=155073;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#6019\6019]8;;\

                    INFO     Creating endpoint with name gender-classifier                          ]8;id=66207;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=947759;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4841\4841]8;;\

---------!